In [1]:
%load_ext autoreload
%autoreload 2

from PIL import Image
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import pickle

In [2]:
# DATA IN CODE HERE

monet = None
vangogh = None
dali = None
picasso = None

with open ('monet.pk', 'rb') as fp:
    monet = pickle.load(fp)
    
with open ('vangogh.pk', 'rb') as fp:
    vangogh = pickle.load(fp)
    
with open ('dali.pk', 'rb') as fp:
    dali = pickle.load(fp)
    
with open ('picasso.pk', 'rb') as fp:
    picasso = pickle.load(fp)
    
    
print(monet[0].shape)


X = np.zeros((1500,32,32,3))
y = np.zeros((1500))

for pic in range(500):
    X[pic] = monet[pic]
    y[pic] = 0
for pic in range(500):
    X[pic + 500] = vangogh[pic]
    y[pic + 500] = 1
for pic in range(500):
    X[pic+ 1000] = dali[pic]
    y[pic + 1000] = 2
'''
for pic in range(100):
    X[pic+ 1500] = picasso[pic]
    y[pic + 1500] = 3
'''
#print(X.shape)
#print(y.shape)

seed = np.random.randint(0, 1000)  

np.random.seed(seed)  
np.random.shuffle(X)  
np.random.seed(seed)  
np.random.shuffle(y)

mean_image = np.mean(X, axis=0)
X -= mean_image

X = X.reshape(1500, -1)
#X_val = X_val.reshape(num_validation, -1)
y = y.astype(int)

num_train = 1400
num_eval = 100

print(num_train, num_eval)

X_train = X[range(num_train)]
X_eval = np.zeros((num_eval, 3072))
y_train = y[range(num_train)]
y_eval = np.zeros(num_eval).astype(int)

for i in range(num_eval):
    X_eval[i] = X[i + num_train]
    y_eval[i] = y[i + num_train]

 

print(X_train.shape)
print(X_eval.shape)
print(y_train[:10])
print(y_eval[:10])

print(X_train[0])

(32, 32, 3)
1400 100
(1400, 3072)
(100, 3072)
[0 2 0 2 2 0 2 0 2 0]
[1 1 0 2 1 2 2 1 1 1]
[112.75133333 107.19333333 101.88266667 ... -39.99466667  -6.972
   2.928     ]


In [3]:
def relu(array):
    return np.maximum(0, array)
def sigmoid(array):
    return 1 / (1 + np.exp(-array))


epoch = 10
learning_rate = .0005
batch_size = 50
N = batch_size
reg = 5e-3

input_size = N
output_size = 4
hidden1 = 16
hidden2 = 16

W1 = np.random.rand(X.shape[1], hidden1) * 1e-2
b1 = np.zeros(hidden1)
W2 = np.random.rand(hidden1, hidden2) * 1e-2
b2 = np.zeros(hidden2)
W3 = np.random.rand(hidden2, output_size) * 1e-2
b3 = np.zeros(output_size)

for i in range(50):
    loss = 0
    for j in range(25):
        
        indices = np.random.choice(X_train.shape[0], batch_size)
        X_curr = X_train[indices]
        y_curr = y_train[indices]
        #print(X_curr)
        
        layer1 = X_curr.dot(W1) + b1
        max_layer1 = np.maximum(0,layer1)
        layer2 = max_layer1.dot(W2) + b2
        max_layer2 = sigmoid(layer2)
        #max_layer2 = np.maximum(0, layer2)
        scores = max_layer2.dot(W3) + b3
        
        exp_scores = np.exp(scores)
        exp_scores_sum = np.sum(exp_scores, axis = 1, keepdims=True)
        softmax = exp_scores / exp_scores_sum
        
        
        loss = np.sum(-np.log(softmax[range(N), y_curr]))
        loss /= N
        loss += reg * (np.sum(W1**2) + np.sum(W2**2) + np.sum(W3**2))
        
        
        
        W3_grads = softmax
        W3_grads[range(X_curr.shape[0]), y_curr] -= 1
        W2_grads = W3_grads.dot(W3.T) * max_layer2 * (1 - max_layer2)
        W1_grads = W2_grads.dot(W2.T) * (layer1 > 0)
        
        gW3 = max_layer2.T.dot(W3_grads) / N
        gb3 = np.sum(W3_grads, axis = 0) / N
        gW2 = max_layer1.T.dot(W2_grads) / N
        gb2 = np.sum(W2_grads, axis = 0) / N
        gW1 = X_curr.T.dot(W1_grads) / N
        gb1 = np.sum(W1_grads, axis = 0) / N
        
        
        gW3 += W3 * reg #* 2
        gW2 += W2 * reg #* 2
        gW1 += W1 * reg #* 2
        
        
        
        W1 += -learning_rate * gW1
        b1 += -learning_rate * gb1
        W2 += -learning_rate * gW2
        b2 += -learning_rate * gb2
        W3 += -learning_rate * gW3
        b3 += -learning_rate * gb3
        
        
        #print(loss)
        #print("\n\n")
    if (i+1) % 5 == 0:
        print("At epoch ", i+1, " the loss is: ", loss)


At epoch  5  the loss is:  1.348273822558491
At epoch  10  the loss is:  1.3138751873117467
At epoch  15  the loss is:  1.2886672519195936
At epoch  20  the loss is:  1.274285144750488
At epoch  25  the loss is:  1.2703718206099535
At epoch  30  the loss is:  1.2487181469012316
At epoch  35  the loss is:  1.2275489730576095
At epoch  40  the loss is:  1.224036337696038
At epoch  45  the loss is:  1.207411106678412
At epoch  50  the loss is:  1.206573013970069


In [4]:
#PREDICT

layer1 = X_eval.dot(W1) + b1
max_layer1 = np.maximum(0, layer1)
layer2 = layer1.dot(W2) + b2
max_layer2 = relu(layer2)
scores = max_layer2.dot(W3) + b3

exp_scores = np.exp(scores)
exp_scores_sum = np.sum(exp_scores, axis = 1, keepdims=True)
softmax = exp_scores / exp_scores_sum

ans = np.argmax(softmax, axis = 1)
#print(softmax)
#print(ans)

correct = 0
for i in range(ans.shape[0]):
    
    if ans[i] == y_eval[i]:
        correct += 1
correct = correct / y_eval.shape[0]
    
print("The Percent Correct is ", correct)

The Percent Correct is  0.48
